# Transpose Upper Triangular Matrix Vector Multiply Routines

This notebook walks you through how to implement $ y := U^T x + y $ where $ U $ is upper triangular.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with `include` and `using` statements.

## Algorithm that takes dot products

## The `Tmvmult_ut_unb_var1!( U, x, y )` routine

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := U^T x + y $.  The "_ut_" in the name of the routine indicates this is the "upper, transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots!( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [9]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Tmvmult_ut_unb_var1!(U, x, y)

    UTL, UTR, 
    UBL, UBR  = flame.part_2x2(U, 
                               0, 0, "TL")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "TOP")

    yT, 
    yB  = flame.part_2x1(y, 
                         0, "TOP")

    while size(UTL, 1) < size(U, 1)

        U00,  u01,       U02,  
        u10t, upsilon11, u12t, 
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, 
                                                         UBL, UBR, 
                                                         1, 1, "BR")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "BOTTOM")

        y0,   
        psi1, 
        y2    = flame.repart_2x1_to_3x1(yT, 
                                        yB, 
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.dots!( upsilon11, chi1, psi1 )
        laff.dots!( u01, x0, psi1 )
        
        #------------------------------------------------------------#

        UTL, UTR, 
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  
                                               u10t, upsilon11, u12t, 
                                               U20,  u21,       U22,  
                                               "TL")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "TOP")

        yT, 
        yB  = flame.cont_with_3x1_to_2x1(y0,   
                                         psi1, 
                                         y2,   
                                         "TOP")

    end

    flame.merge_2x1!(yT, 
                     yB, y)

end

Tmvmult_ut_unb_var1! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [10]:
U = rand(4, 4)
x = rand(4)
y = rand(4)
yold = rand(4)

# Notice that U is not upper triangular.  We will only use the upper triangular part.

println( "U before =" )
U

U before =


4×4 Array{Float64,2}:
 0.94883   0.384147  0.00294439  0.782414
 0.533379  0.12194   0.425606    0.193788
 0.510094  0.79745   0.144869    0.210651
 0.560685  0.579221  0.240718    0.820349

In [11]:
println( "x before =" )
x

x before =


4-element Array{Float64,1}:
 0.7946370646671832
 0.8740745452831999
 0.9806021236294933
 0.2545552268874598

In [12]:
println( "y before =" )
y

y before =


4-element Array{Float64,1}:
 0.6409003832238664 
 0.38285213960168174
 0.9422115549836747 
 0.4333447490865663 

In [13]:
laff.copy!( y, yold )   # save the original vector y

Tmvmult_ut_unb_var1!( U, x, y )

println( "y after =" )
y

y after =


4-element Array{Float64,1}:
 1.3948760233387052
 0.7946944292429243
 1.4586219118368198
 1.639854371600785 

In [14]:
using LinearAlgebra
# UpperTriangular() makes the matrix upper triangular
println( "y - ( transpose( UpperTriangular( U ) ) * x + yold ) = " )
y - ( transpose( UpperTriangular( U ) ) * x + yold )

y - ( transpose( UpperTriangular( U ) ) * x + yold ) = 


4-element Array{Float64,1}:
  0.0                  
  0.0                  
  0.0                  
 -2.220446049250313e-16

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

## The `Tmvmult_ut_unb_var2!( U, x, y )` routine

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := U^T x + y $.  The "_ut_" in the name of the routine indicates this is the "upper triangular, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy!( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [15]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Tmvmult_ut_unb_var2!(U, x, y)

    UTL, UTR, 
    UBL, UBR  = flame.part_2x2(U, 
                               0, 0, "TL")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "TOP")

    yT, 
    yB  = flame.part_2x1(y, 
                         0, "TOP")

    while size(UTL, 1) < size(U, 1)

        U00,  u01,       U02,  
        u10t, upsilon11, u12t, 
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, 
                                                         UBL, UBR, 
                                                         1, 1, "BR")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "BOTTOM")

        y0,   
        psi1, 
        y2    = flame.repart_2x1_to_3x1(yT, 
                                        yB, 
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.axpy!( chi1, u12t, y2 )
        laff.axpy!( chi1, upsilon11, psi1 )
        
        #------------------------------------------------------------#

        UTL, UTR, 
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  
                                               u10t, upsilon11, u12t, 
                                               U20,  u21,       U22,  
                                               "TL")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "TOP")

        yT, 
        yB  = flame.cont_with_3x1_to_2x1(y0,   
                                         psi1, 
                                         y2,   
                                         "TOP")

    end

    flame.merge_2x1!(yT, 
                     yB, y)

end

Tmvmult_ut_unb_var2! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [16]:
U = rand(4, 4)
x = rand(4)
y = rand(4)
yold = rand(4)

# Notice that U is not upper triangular.  We will only use the upper triangular part.

println( "U before =" )
U

U before =


4×4 Array{Float64,2}:
 0.0748038  0.819176  0.505906   0.436579
 0.324334   0.710495  0.676943   0.54178 
 0.180601   0.548393  0.0870379  0.466402
 0.878547   0.747691  0.0689026  0.168852

In [17]:
println( "x before =" )
x

x before =


4-element Array{Float64,1}:
 0.38375143719773885
 0.3896637942871266 
 0.03955462436444801
 0.2267630254014723 

In [18]:
println( "y before =" )
y

y before =


4-element Array{Float64,1}:
 0.8446878569286851  
 0.5718373118183055  
 0.017118971943671735
 0.24454998535829198 

In [19]:
laff.copy!( y, yold )   # save the original vector y

Tmvmult_ut_unb_var2!( U, x, y )

println( "y after =" )
y

y after =


4-element Array{Float64,1}:
 0.8733939211184157
 1.1630514682114743
 0.4784837397100513
 0.6799377152374568

In [20]:
using LinearAlgebra
# UpperTriangular() makes the matrix upper triangular
println( "y - ( transpose( UpperTriangular( U ) ) * x + yold ) = " )
y - ( transpose( UpperTriangular( U ) ) * x + yold )

y - ( transpose( UpperTriangular( U ) ) * x + yold ) = 


4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.